## Test obecného fine-tuningu

In [ ]:
# --- 1. Konfigurace ---
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset

# Cesta k vašemu lokálně uloženému modelu
MODEL_PATH = "/home/luk-hrd/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/63a8b081895390a26e140280378bc85ec8bce07a"

# Název datasetu na Hugging Face Hubu
DATASET_NAME = "databricks/databricks-dolly-15k"

# Název pro váš nový, fine-tunovaný model (pro výstupní adresář)
NEW_MODEL_NAME = "mistral-7b-quantum-instruct-test"

In [2]:
# --- 2. Načtení a příprava modelu pro LoRA trénink ---

# A. Konfigurace pro 4-bitovou kvantizaci (QLoRA)
# Toto je klíčové pro snížení paměťové náročnosti
print("Vytváření 4-bitové konfigurace (QLoRA)...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# B. Načtení základního modelu s kvantizací
print(f"Načítání základního modelu: {MODEL_PATH}...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto"  # Automaticky rozmístí model na dostupné GPU
)

# C. Načtení tokenizeru a nastavení padding tokenu
# U kauzálních modelů je běžné nastavit pad_token na eos_token
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token

# D. Příprava kvantizovaného modelu pro trénink
# Zapne gradient checkpointing a další optimalizace
model = prepare_model_for_kbit_training(model)

# E. Vytvoření LoRA konfigurace
# Zde definujeme, jak budou vypadat naše malé adaptéry
print("Vytváření LoRA konfigurace...")
lora_config = LoraConfig(
    r=8,  # Rank - nižší hodnota = méně parametrů, doporučeno 8, 16, 32
    lora_alpha=16, # Scaling faktor, často dvojnásobek 'r'
    # Řekneme, na které vrstvy modelu se mají LoRA adaptéry připojit.
    # Pro Mistral/Llama jsou toto standardní volby.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# F. Aplikace LoRA na model
# Tímto příkazem se model "obalený" LoRA adaptéry stane PEFT modelem
model = get_peft_model(model, lora_config)

# G. (Doporučeno) Ověření a výpis trénovatelných parametrů
# Tato pomocná funkce vám ukáže, jak málo parametrů se reálně trénuje
def print_trainable_parameters(model):
    """Prints the number of trainable parameters in the model."""
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

print_trainable_parameters(model)


# --- 3. Načtení datasetu ---
# Tato část zůstává beze změny
print("Načítání datasetu...")
dataset = load_dataset(DATASET_NAME)

Vytváření 4-bitové konfigurace (QLoRA)...
Načítání základního modelu: /home/luk-hrd/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapshots/2dcff66eac0c01dc50e4c41eea959968232187fe...


Loading checkpoint shards: 100%|██████████| 2/2 [00:28<00:00, 14.18s/it]


Vytváření LoRA konfigurace...
trainable params: 6815744 || all params: 3758886912 || trainable%: 0.18
Načítání datasetu...


In [ ]:
# --- 4. Příprava dat ---
# Set pad_token to eos_token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    # Combine instruction, context, and response into one string
    texts = [
        f"Instruction: {instr}\nContext: {ctx}\nResponse: {resp}"
        for instr, ctx, resp in zip(examples["instruction"], examples["context"], examples["response"])
    ]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=512)

print("Předzpracování dat...")
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# --- 5. Konfigurace trénování ---
training_args = TrainingArguments(
    output_dir=NEW_MODEL_NAME,
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir=f"{NEW_MODEL_NAME}/logs",
    logging_steps=10,
    save_total_limit=2,
    fp16=True,  # Použití mixed precision (pokud je dostupné)
    push_to_hub=False  # Nastavte na True, pokud chcete model nahrát na Hugging Face Hub
)

Předzpracování dat...


In [14]:
# Místo celého datasetu si vezměte jen prvních 1000 příkladů
small_train_dataset = tokenized_dataset["train"].select(range(100))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset, # <<< Použijte menší dataset
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipykernel_30841/233178323.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [15]:
# --- 5. Konfigurace trénování ---
training_args = TrainingArguments(
    output_dir=NEW_MODEL_NAME,
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir=f"{NEW_MODEL_NAME}/logs",
    logging_steps=10,
    save_total_limit=2,
    fp16=True,  # Použití mixed precision (pokud je dostupné)
    push_to_hub=False,  # Nastavte na True, pokud chcete model nahrát na Hugging Face Hub
    gradient_checkpointing=False, # <<< TOTO PŘIDEJTE
)


In [16]:
from transformers import Trainer, DataCollatorForLanguageModeling

# --- 5. Vytvoření Data Collatoru ---
# Data Collator je zodpovědný za přípravu dávky dat pro trénink.
# V tomto případě použijeme DataCollatorForLanguageModeling, který je vhodný pro jazykové modely.
# Nastavíme mlm=False, protože trénujeme kauzální jazykový model (např. Mistral, Llama).
print("Vytváření Data Collatoru...")
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False
)


# --- 6. Fine-tuning ---
# Teď předáme vytvořený data_collator do Traineru
print("Spouštění fine-tuningu...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator  # <<< TOTO JE KLÍČOVÁ ZMĚNA
)

# Tato část se spouští až po úspěšné inicializaci
trainer.train()

/tmp/ipykernel_30841/2267052994.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Vytváření Data Collatoru...
Spouštění fine-tuningu...


/home/luk-hrd/miniconda3/envs/mistral-lora/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.964100
20,1.724400
30,1.802800
40,1.890300
50,2.028200
60,1.683800
70,1.757200
80,1.702900
90,2.109900
100,1.640000


/home/luk-hrd/miniconda3/envs/mistral-lora/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/luk-hrd/miniconda3/envs/mistral-lora/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, 

TrainOutput(global_step=300, training_loss=1.6402534612019857, metrics={'train_runtime': 1843.8989, 'train_samples_per_second': 0.163, 'train_steps_per_second': 0.163, 'total_flos': 6559465734144000.0, 'train_loss': 1.6402534612019857, 'epoch': 3.0})

In [17]:
# --- 7. Uložení modelu ---
print("Ukládání modelu...")
model.save_pretrained(NEW_MODEL_NAME)
tokenizer.save_pretrained(NEW_MODEL_NAME)

print(f"Fine-tuning dokončen. Model uložen v: {NEW_MODEL_NAME}")

Ukládání modelu...
Fine-tuning dokončen. Model uložen v: mistral-7b-quantum-instruct-test
